In [ ]:
!pip install openai langchain moviepy pydub py-llm-core pytube

# Download Video


In [4]:
from ipywidgets import widgets
from IPython.display import display
from pytube import YouTube

text = widgets.Text(
    value='',
    placeholder='Enter YouTube URL',
    description='YouTube URL:',
)

button = widgets.Button(description="Download")

def download_action(b):
    link = text.value
    youtubeObject = YouTube(link)
    youtubeObject = youtubeObject.streams.get_highest_resolution()
    try:
        youtubeObject.download()
        print("Download is completed successfully")
    except Exception as e:
        print(f"An error has occurred: {e}")

button.on_click(download_action)

display(text, button)

Text(value='', description='YouTube URL:', placeholder='Enter YouTube URL')

Button(description='Download', style=ButtonStyle())

Download is completed successfully
Download is completed successfully


Mount your google drive so files can be saved and retrieved permanently

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Extract and save audio from video stored in google drive

In [6]:
from moviepy.editor import *

# Path to the video file on Google Drive
video_path = "/content/function_calling.mp4"

# Load the video
video = VideoFileClip(video_path)

# Extract audio
audio = video.audio

# Path to save the output audio file
audio_output_path = "/content/function_calling.mp3"

# Save audio as MP3
audio.write_audiofile(audio_output_path)

# Close the clips
video.close()
audio.close()

MoviePy - Writing audio in /content/function_calling.mp3


MoviePy - Done.


whisper can only accept files of up to 25 mb, so a function is needed to check if the files need to be split up.

Import Libraries and Define the check_size Function

In [10]:
import os
from pydub import AudioSegment
import openai

# Function to check file size in MB
def check_size(file_path):
    return os.path.getsize(file_path) / (1024 * 1024)

 Load and Optionally Compress Audio File

In [13]:
# Initialize paths and list to store files for transcription
audio_output_path = "/content/function_calling.mp3"
compressed_audio_path = "/content/compressed_audio.mp3"
files_to_transcribe = []

# Check initial file size
initial_size = check_size(audio_output_path)

# Load original audio file
audio = AudioSegment.from_file(audio_output_path)

# Check if compression is needed
if initial_size > 25:
    audio = audio.set_frame_rate(16000).set_channels(1)
    audio.export(compressed_audio_path, format="mp3", bitrate="48k")
    compressed_size = check_size(compressed_audio_path)
else:
    compressed_audio_path = audio_output_path
    compressed_size = initial_size

Optionally Chunk the Audio File

In [15]:
# Check if chunking is needed
if compressed_size > 25:
    num_chunks = int(compressed_size // 25) + 1
    total_length = len(audio)
    chunk_length = total_length // num_chunks

    for i in range(num_chunks):
        start_point = i * chunk_length
        end_point = (i + 1) * chunk_length if i != num_chunks - 1 else total_length
        chunk = audio[start_point:end_point]
        chunk_path = f"/content/chunk_{i+1}.mp3"
        chunk.export(chunk_path, format="mp3")
        files_to_transcribe.append(chunk_path)
else:
    files_to_transcribe.append(compressed_audio_path)

# Whisper Transcription

In [35]:
import openai
from ipywidgets import Password, Button, HBox, VBox
from IPython.display import display

# Create widgets
api_key_input = Password(description='API Key:', placeholder='Enter your OpenAI API key')
submit_button = Button(description='Submit')

# Define function to execute on button press
def store_api_key(b):
    openai.api_key = api_key_input.value
    print("API Key set!")

# Link button press to function
submit_button.on_click(store_api_key)

# Display widgets
display(HBox([api_key_input, submit_button]))

# Your existing code below
# Initialize the transcripts dictionary
transcripts = {}

# Loop to transcribe files
for idx, file_path in enumerate(files_to_transcribe):
    with open(file_path, "rb") as audio_file:
        transcripts[f"transcript_{idx+1}"] = openai.Audio.transcribe("whisper-1", audio_file)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-35-774d05a274b2>", line 27, in <cell line: 25>
    transcripts[f"transcript_{idx+1}"] = openai.Audio.transcribe("whisper-1", audio_file)
  File "/usr/local/lib/python3.10/dist-packages/openai/api_resources/audio.py", line 72, in transcribe
    response, _, api_key = requestor.request("post", url, files=files, params=data)
  File "/usr/local/lib/python3.10/dist-packages/openai/api_requestor.py", line 289, in request
    result = self.request_raw(
  File "/usr/local/lib/python3.10/dist-packages/openai/api_requestor.py", line 606, in request_raw
    result = _thread_context.session.request(
  File "/usr/local/lib/python3.10/dist-packages/requests/sessions.py", line 589, in request
    resp = self.send(prep, **send_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/reque

TypeError: ignored

API Key set!


In [19]:
print(transcripts)

{'transcript_1': <OpenAIObject at 0x7d86724e0810> JSON: {
  "text": "Let's talk about GPT function calling. I think of GPT as this kind of brain that's floating out there in the cloud somewhere, and you can talk to it. You can talk to it through apps like ChatGPT, but you can also write code that talks to it through the API. So GPT functions were a recent addition, which are, well, they're super useful, but a little bit hard to wrap your head around sometimes. So I hope this video will make it crystal clear and you'll understand what it is and what you can use it for. So we're going to be doing some code examples. So here's our basic scaffolding, right? Some JavaScript, open AI key comes in there. We select a model there, and then we ask GPT this question and we log the answer. So let's try it. There we go. There's our generic, long, boring AI answer, but it's working. So the question is, what if we write something like this? Is it raining in Stockholm? So there we go. We get a long an

 Combine Transcripts into a Master Transcript

In [21]:
# Initialize an empty string to store the master transcript
master_transcript = ""

# Loop through all keys in the 'transcripts' dictionary
for idx, transcript_key in enumerate(transcripts.keys()):
    # Extract the transcribed text stored under the key 'text_content'
    transcript_text = transcripts[transcript_key]['text']

    # Append the transcript_text to master_transcript
    master_transcript += transcript_text + " "

# Optionally, write the master transcript to a text file
with open("/content/master_transcript.txt", "w") as f:
    f.write(master_transcript)

In [22]:
print(master_transcript)

Let's talk about GPT function calling. I think of GPT as this kind of brain that's floating out there in the cloud somewhere, and you can talk to it. You can talk to it through apps like ChatGPT, but you can also write code that talks to it through the API. So GPT functions were a recent addition, which are, well, they're super useful, but a little bit hard to wrap your head around sometimes. So I hope this video will make it crystal clear and you'll understand what it is and what you can use it for. So we're going to be doing some code examples. So here's our basic scaffolding, right? Some JavaScript, open AI key comes in there. We select a model there, and then we ask GPT this question and we log the answer. So let's try it. There we go. There's our generic, long, boring AI answer, but it's working. So the question is, what if we write something like this? Is it raining in Stockholm? So there we go. We get a long answer, basically boiling down to screw you. No, I will not do this. In

# Chain of density

In [28]:
# implement chain of density prompting
from typing import List
from dataclasses import dataclass
from llm_core.assistants import OpenAIAssistant

@dataclass
class DenseSummary:
    denser_summary: str
    missing_entities: List[str]


@dataclass
class DenserSummaryCollection:
  system_prompt = """
  You are an expert in writing rich and dense summaries in broad domains.
  """

  prompt = """

  Context:

  The following speech is a youtube lecture on Open AI function calling.


  ----
  Speech:

  {article}

  ----

  You will generate increasingly concise, entity-dense summaries of the speech snippet. You will use any context provided to make sense of and orient the
  summary of the speech with what has come before.

  Repeat the following 2 steps 5 times.

  - Step 1: Identify 1-3 informative Entities from the Article
  which are missing from the previously generated summary and are the most
  relevant.

  - Step 2: Write a new, denser summary of identical length which covers
  every entity and detail from the previous summary plus the missing entities

  A Missing Entity is:

  - Relevant: to the main story
  - Specific: descriptive yet concise (5 words or fewer)
  - Novel: not in the previous summary
  - Faithful: present in the Article
  - Anywhere: located anywhere in the Article

  Guidelines:
  - Use the context provided to make sense of the article and ensure that there
  is consistency between the information provided in the context and the
  information provided in the article.

  - Do not repeat points made in the speech if they are already provided in the context.

  - Only add new pieces of information, make sure nothing mentioned in the context is repeated.

  - The first summary should be long (4-5 sentences, approx. 80 words) yet
  highly non-specific, containing little information beyond the entities
  marked as missing.

  - Use overly verbose language and fillers (e.g. "this article discusses") to
  reach approx. 80 words.

  - Make every word count: re-write the previous summary to improve flow and
  make space for additional entities.

  - Make space with fusion, compression, and removal of uninformative phrases
  like "the article discusses"

  - The summaries should become highly dense and concise yet self-contained,
  e.g., easily understood without the Article.

  - Missing entities can appear anywhere in the new summary.

  - Never drop entities from the previous summary. If space cannot be made,
  add fewer new entities.

  > Remember to use the exact same number of words for each summary.
  Answer in JSON.

  > The JSON in `summaries_per_step` should be a list (length 5) of
  dictionaries whose keys are "missing_entities" and "denser_summary".

  """

  summaries: List[DenseSummary]


  @classmethod
  def summarize(cls, article):
      with OpenAIAssistant(cls, model='gpt-4') as assistant:
          return assistant.process(article=article)

In [29]:
summary = DenserSummaryCollection.summarize(master_transcript)


DenserSummaryCollection(summaries=[DenseSummary(denser_summary="The speaker discusses the concept of GPT function calling in a YouTube lecture. He explains how GPT functions can be used to enhance the capabilities of GPT, such as asking about the weather in a specific city. The speaker also demonstrates how to write code that interacts with GPT through the API. He further illustrates how to define functions and how GPT can use these functions to generate responses. The speaker also highlights the importance of the sandbox in GPT, which prevents it from accessing the internet or the user's file system.", missing_entities=['YouTube lecture', 'GPT function calling', 'API']), DenseSummary(denser_summary="In a YouTube lecture, the speaker elucidates GPT function calling, demonstrating how to use the API to enhance GPT's capabilities. He shows how to define functions like 'getWeather', which GPT can utilize to generate responses. He emphasizes the sandbox's role in GPT, preventing it from ac

In [34]:
print(summary.summaries[4].denser_summary)